## Basic prompty sample
- Chat with prompty
- Add weather function
- http request using natural language
- code interpreter (powershell)

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.22.0"
#r "nuget: Microsoft.SemanticKernel.Prompty, 1.22.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.22.0"

#r "nuget:AutoGen,0.2.1"
#r "nuget:AutoGen.DotnetInteractive,0.2.1"

In [ ]:
Start-Process pwsh -ArgumentList "-c dotnet run --project .\Interactive.Prompty\WeatherService\WeatherForecastingService.csproj"

In [ ]:
#r "Interactive.Prompty/Interactive.Prompty/bin/Debug/net8.0/Interactive.Prompty.dll"

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using System.ComponentModel;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Events;
using Microsoft.DotNet.Interactive.Commands;
using SKKernel = Microsoft.SemanticKernel.Kernel;
using Kernel = Microsoft.DotNet.Interactive.Kernel;
using Interactive.Prompty;
using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using AutoGen.DotnetInteractive;
using AutoGen.DotnetInteractive.Extension;


await PromptyExtension.LoadExtensionAsync((CompositeKernel)Microsoft.DotNet.Interactive.Kernel.Root);

In [ ]:
#!connect prompty --kernel-name prompty

In [ ]:
---
name: ExamplePrompt
description: A prompt that uses context to ground an incoming question
authors:
  - HeHe
model:
  api: chat
sample:
  firstName: Geno
  context: empty
---
 
system:
You are an AI assistant who helps people with_anything. As the assistant,
you answer questions briefly, succinctly, and in a personable manner using markdown and even add some personal flair with appropriate emojis. You try to resolve customer's question in one-step

The following http services are available. You can make request to these services using http request syntax

## weather forecast service
- base address: http://localhost:5049
- swagger
```json
{
    "openapi": "3.0.1",
    "info": {
        "title": "WeatherForecastingService",
        "version": "1.0"
    },
    "paths": {
        "/weatherforecast": {
            "get": {
                "tags": [
                    "WeatherForecastingService"
                ],
                "operationId": "GetWeatherForecast",
                "responses": {
                    "200": {
                        "description": "OK",
                        "content": {
                            "application/json": {
                                "schema": {
                                    "type": "array",
                                    "items": {
                                        "$ref": "#/components/schemas/WeatherForecast"
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    },
    "components": {
        "schemas": {
            "WeatherForecast": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "format": "date"
                    },
                    "temperatureC": {
                        "type": "integer",
                        "format": "int32"
                    },
                    "summary": {
                        "type": "string",
                        "nullable": true
                    },
                    "temperatureF": {
                        "type": "integer",
                        "format": "int32",
                        "readOnly": true
                    }
                },
                "additionalProperties": false
            }
        }
    }
}
```
 
# Context
Use the following context to provide a more personalized response to {{firstName}}:
{{context}}
 
user:
{{input}}

In [ ]:
#!connect prompty-orchestrator --kernel-name orchestrator-kernel --prompty-kernel-name prompty --azure-openai-endpoint @input:{"saveAs":"azure-openai-endpoint"} --azure-openai-deployment-name @input:{"saveAs":"azure-openai-deployment-name"} --azure-openai-apikey @password:{"saveAs":"azure-openai-apikey"}

In [ ]:
Who are you helping with

In [ ]:
[KernelFunction]
[Description("Run Powershell")]
public async static Task<string> Powershell([Description("the powershell script to run")] string pwsh)
{
    var pwshKernel = Kernel.Root.FindKernelByName("pwsh");
    $"generated code {pwsh}".Display();
    var command = new SubmitCode(pwsh);
    List<KernelEvent> events = new List<KernelEvent>();
    var sub = Kernel.Root.KernelEvents.Subscribe(e => events.Add(e));
    var result = await pwshKernel.SendAsync(command);   
    sub.Dispose();
    var displayValues = events
                        .OfType<DisplayEvent>()
                        .SelectMany(x => x.FormattedValues);

    var response =  string.Join("\n", displayValues.Select(x => x.Value)) ?? "Nothing to display";
    $"results : {response}".Display();
    return response;
}

[KernelFunction]
[Description("Run http request")]
public async static Task<string> Http([Description("the HTTP request to run")] string http)
{
    var httpKernel = Kernel.Root.FindKernelByName("http"); 
    $"generated code {http}".Display();
    var command = new SubmitCode(http);
    List<KernelEvent> events = new List<KernelEvent>();
    var sub = Kernel.Root.KernelEvents.Subscribe(e => events.Add(e));
    var result = await httpKernel.SendAsync(command);
    sub.Dispose();
    var displayValues = events
                        .OfType<DisplayEvent>()
                        .SelectMany(x => x.FormattedValues);

    var response = string.Join("\n", displayValues.Select(x => x.Value)) ?? "Nothing to display";
    $"results : {response}".Display();
    return response;
}

In [ ]:
show me the how many files are in current folder using powershell, list their names and their size in bytes

In [ ]:
use http to create a weather forecasting report for me